# Elective Module Advanced Topics of Data Mining.
---
<b>MADS-EMDM Portfolio-Exam Part 1<br>
Janosch Höfer, 938969</b>

## Table of contents

- [Introduction](#intro) <br>
- [1. Exercise](#ex1) <br>
- [2. Exercise](#ex2) <br>
- [3. Exercise](#ex3) <br>
- [4. Exercise](#ex4) <br>
- [5. Exercise](#ex5)<br>
- [References](#ref)

## Introduction

bla<br>
Using [[1]](http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions)

In [ ]:
# Standard libraries
import functools
import itertools
import os
import timeit

# Installed libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

# Own classes and functions
from helper_scripts.data_manipulation import setup_raw_data

---
<a id='ex1'></a>

## 1. Exercise

Consider the following set of items: a, b, c, d, e, g, m in some transaction database. Assume that
<b>APRIORI</b> is running and has already computed the set 𝐿4 using alphabetical order on the items. The
resulting set is:
$$
L_4 = \begin{equation}
\left\{
    \begin{aligned}
        \{a, b, c, d\}, \{a, b, c, e\}, \{a, b, c, m\}, \{a, b, d, e\}, \{a, b, e, m\} \\
        \{a, c, e, m\}, \{a, d, g, m\}, \{b, c, d, e\}, \{b, c, d, g\}, \{b, c, d, m\} \\
        \{b, c, e, g\}, \{b, c, e, m\}, \{b, d, e, g\}, \{c, d, e, g\}, \{c, d, g, m\} \\
    \end{aligned}
    \right\}
\end{equation}
$$

### 1.1. Use the APRIORI candidate generation (and alphabetical order on the items) to create the set $C_5$ of candidates for 5-element frequent itemsets. Explain your decisions to create or discard elements.

bla
$$
C_5 = \begin{equation}
\left\{
    \begin{aligned}
        \{a, b, c, d, e\}, \{b, c, d, e, g\}, \{b, c, d, e, m\}, \{b, c, d, g, m\}, \{b, c, e, g, m\}
    \end{aligned}
    \right\}
\end{equation}
$$

### 1.2. For how many itemsets does the database have to be scanned to yield $L_5$ from $C_5$?

Only 1, because all the other itemsets can be removed beforehand,"because at least one of their sets is not present in $C_5$.
$$
L_5 = \begin{equation}
\left\{
    \begin{aligned}
        \{b, c, d, e, m\}
    \end{aligned}
    \right\}
\end{equation}
$$

---
<a id='ex2'></a>

## 2. Exercise

### 2.1. Create the following grocery dataset:

In [ ]:
dataset1 = [
    [
        "fish",
        "apples",
        "cider",
        "dragon fruit",
        "garlic",
        "ice cream",
        "mints",
        "prunes",
    ],
    ["apples", "bacon", "cider", "fish", "lemons", "mints", "oatmeal"],
    ["bacon", "fish", "ham", "jam", "oatmeal"],
    ["bacon", "cider", "kiwis", "spam", "prunes"],
    ["apples", "fish", "cider", "eggs", "lemons", "prunes", "mints", "nachos"],
]

### 2.2. Load the dataset T10I4D100K from the Frequent Itemset Mining Dataset Repository.

In [ ]:
# Download data
filename_data = "T10I4D100K.dat"
data_url = "http://fimi.uantwerpen.be/data"
path_to_data = "data"

setup_raw_data(f"{data_url}/{filename_data}", path_to_data, filename_data)

After downloading the data file, we must first bring it into the right format. First we split each row of text. Here it is important to remove the last entry in the list which is empty. This empty entry is the result of the method used, as can be seen in the example below:

In [ ]:
"test\n".split("\n")

In [ ]:
with open(os.path.join(path_to_data, filename_data), "r") as file:
    dataset2_raw = file.read().split("\n")[:-1]

Next we split each row into the items.

In [ ]:
dataset2_raw[-2:]

In [ ]:
dataset2_long = [item.rstrip().split(" ") for item in dataset2_raw]
len(dataset2_long)

In [ ]:
dataset2 = dataset2_long[:10000]
len(dataset2)

### 2.3. For each dataset compute the highest (relative) support that a non-empty itemset actually reaches.

Before we can calculate the support, we must encode the transactions. For that we use the TransactionEncoder provided by the <i>mlxtend</i> library.

In [ ]:
def encode_data(dataset: list[list], sparse: bool = False) -> pd:
    te = TransactionEncoder()
    te_ary = te.fit_transform(dataset, sparse=sparse)
    if sparse:
        return pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)
    return pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
data1 = encode_data(dataset1)
data2 = encode_data(dataset2)

With the encoded transactions, we are able to count the number of occurrences.

In [ ]:
data1.head()

In [ ]:
relsup_str = "relativ_support"
pd.DataFrame(data1.sum() / data1.shape[0], columns=[relsup_str]).nlargest(5, relsup_str)

The highest relative support for dataset1 is 0.8 for the itemsets $\{cider\}$ and $\{fish\}$.

In [ ]:
pd.DataFrame(data2.sum() / data2.shape[0], columns=[relsup_str]).nlargest(5, relsup_str)

The highest relative support for dataset2 is 0.07828 for the itemset $\{368\}$.

---
<a id='ex3'></a>

## 3. Exercise

### 3.1. Run and time APRIORI once on the grocery dataset using a support threshold of 0.4 and a fitting line magic command. How long did the run take? How many frequent itemsets were found?

In [ ]:
%%timeit -o -q -n 1 -r 1

df1_apriori = apriori(data1, min_support=0.4, use_colnames=True)
print(f"The command found {df1_apriori.shape[0]} frequent itemsets.")

In [ ]:
print(f"And it ran for {_.average * 100:0.2f} milliseconds.")

### 3.2. Run and time APRIORI once on the T10I4D100K dataset using a support threshold of 0.004 and a fitting line magic command. How long did the run take? How many frequent itemsets were found?

In [ ]:
%%timeit -o -q -n 1 -r 1
df2_apriori = apriori(data2, min_support=0.004, use_colnames=True)
print(f"The command found {df2_apriori.shape[0]} frequent itemsets.")

In [ ]:
print(f"And it ran for {_.average:0.5f} seconds.")

### 3.3. Time repeated runs of APRIORI on both datasets using the above respective support thresholds and a fitting line magic command where:

- on the grocery dataset, 10 measurements are taken, each relying on 10 executions
- on the T10I4D100K dataset, 10 measurements are taken, each relying on 1 execution.

Report average runtime together with standard deviation for both experiments.

In [ ]:
timeit_results = dict()

In [ ]:
%timeit -o -n 10 -r 10 apriori(data1, min_support=0.4, use_colnames=True)

In [ ]:
timeit_results["Grocery_Dataset"] = _

In [ ]:
%timeit -o -n 1 -r 10 apriori(data2, min_support=0.004, use_colnames=True)

In [ ]:
timeit_results["T10I4D100K"] = _

In [ ]:
for key, timit_res in timeit_results.items():
    print(f"Results for '{key}':")
    print(
        f"The command ran on average for {timit_res.average:0.5f}"
        f" seconds and has a standard deviation of {timit_res.stdev:0.10f}"
    )

#### 3.4. Discuss limitations of the above approaches!

- Terrible to use with a good linter

---
<a id='ex4'></a>

## 4. Exercise

### 4.1. Time and run APRIORI on both datasets, first on the regular (non-sparse) representation, then on a sparse representation.

- For the grocery dataset use 0.2, 0.4, 0.6, 0.8 as the support thresholds, run 10 measurements for each setting and use 10 executions for each measurement.
- For the T10I4D100K dataset use 0.002, 0.004, 0.008, 0.016, 0.032, 0.064 as the support thresholds, run 10 measurements for each setting and use 1 execution for each repetition.

Out of the 10 measurements, report only the minimum for each setting (column parameter and
support threshold).

In [ ]:
def run_and_time(func: callable, n_mes: int, n_exec: int, func_kwargs: dict) -> tuple[int, float]:
    part_func = functools.partial(func, **func_kwargs)
    results = timeit.repeat(part_func, repeat=n_mes, number=n_exec)
    min_time = min(results) * 100
    memory = part_func().memory_usage().sum()
    return min_time, memory


def experiment_maker(
    dataset: list[list], func: callable, parameters: dict[str:list]
) -> pd.DataFrame:
    results = list()
    max_len = np.prod([len(item) for item in parameters.values()])
    for item in tqdm(itertools.product(*parameters.values()), total=max_len):
        data = encode_data(dataset, sparse=item[3])
        avg, memory = run_and_time(
            func,
            item[1],
            item[2],
            {"df": data, "min_support": item[0], "use_colnames": item[4]},
        )
        results.append(
            [
                *item,
                avg,
                data.memory_usage().sum(),
                memory,
            ]
        )
    run_resultnames = ["min_time [ms]", "memory_encoding", "memory_itemsets"]
    df_colnames = list(parameters.keys()) + run_resultnames
    return pd.DataFrame(results, columns=df_colnames)

In [ ]:
# Setup parameters
data1_params_sparse = {
    "min_supports": [0.2, 0.4, 0.6, 0.8],
    "measurements": [10],
    "executions": [10],
    "sparse": [False, True],
    "colnames": [False],
}
data2_params_sparse = {
    "min_supports": [0.002, 0.004, 0.008, 0.016, 0.032, 0.064],
    "measurements": [10],
    "executions": [1],
    "sparse": [False, True],
    "colnames": [False],
}

In [ ]:
%%time
res_1_apri = experiment_maker(dataset1, apriori, data1_params_sparse)

In [ ]:
%%time
res_2_apri = experiment_maker(dataset2, apriori, data2_params_sparse)

### 4.2. Display results on each dataset in a separate, suitable table (create useful columns), focussing on the comparison of runtime and memory consumption for executions on non-sparse and sparse data structures.

In [ ]:
res_1_apri

In [ ]:
res_2_apri

### 4.3. Interpret the results – state observations, limitations, advice regarding the use of sparse representations!


### 4.4. Explain, why only the minimum runtime is reported (in contrast to other options, like the average runtime).



---
<a id='ex5'></a>

## 5. Exercise

### 5.1. Time and run APRIORI and FP-Growth on both datasets.
 - For the grocery dataset use 0.2, 0.4, 0.6, 0.8 as the support thresholds, run 10 measurements for each setting and use 10 executions for each repetition, and
 - for the T10I4D100K dataset use 0.002, 0.004, 0.008, 0.016, 0.032, 0.064 as the support thresholds, run 10 measurements for each setting and use 1 execution for each repetition.

Out of the 10 measurements, report only the minimum for each setting (column parameter and
support threshold).

In [ ]:
# Setup parameters
data1_params = {
    "min_supports": [0.2, 0.4, 0.6, 0.8],
    "measurements": [10],
    "executions": [10],
    "sparse": [False],
    "colnames": [False],
}
data2_params = {
    "min_supports": [0.002, 0.004, 0.008, 0.016, 0.032, 0.064],
    "measurements": [10],
    "executions": [1],
    "sparse": [False],
    "colnames": [False],
}

In [ ]:
algo_dict = {"apriori": apriori, "fpgrowth": fpgrowth}

In [ ]:
%%time
res_1 = list()
for key, func in algo_dict.items():
    _df = experiment_maker(dataset1, func, data1_params)
    _df["Algorithm"] = key
    res_1.append(_df)

In [ ]:
%%time
res_2 = list()
for key, func in algo_dict.items():
    _df = experiment_maker(dataset2, func, data2_params)
    _df["Algorithm"] = key
    res_2.append(_df)

### 5.2.Display results on each dataset in a separate, suitable table (create useful columns), focussing on the comparison of runtime for executions of the two algorithms.

In [ ]:
table_1 = pd.concat(res_1)
table_1.columns = table_1.columns.str.capitalize()
table_1[["Algorithm", "Min_supports", "Min_time [ms]", "Memory_encoding", "Memory_itemsets"]]

In [ ]:
table_2 = pd.concat(res_2)
table_2.columns = table_2.columns.str.capitalize()
table_2[["Algorithm", "Min_supports", "Min_time [ms]", "Memory_encoding", "Memory_itemsets"]]

### 5.3. Add a visual comparison by plotting the runtimes of both algorithms against the support thresholds into the same diagram (thus, one diagram per dataset). Use scaled axes where useful.

In [ ]:
fig, ax = plt.subplots()
ax = sns.lineplot(data=table_1, x="Min_supports", y="Min_time [ms]", hue="Algorithm")
ax.set(
    xlabel="Support Thresholds",
    ylabel="Minimum runtime [ms]",
    title="Runtime comparison of 'Apriori' and 'fpgrowth' algorithm for different supports",
)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax = sns.lineplot(data=table_2, x="Min_supports", y="Min_time [ms]", hue="Algorithm")
ax.set(
    xlabel="Support Thresholds",
    ylabel="Minimum runtime [ms]",
    title="Runtime comparison of 'Apriori' and 'fpgrowth' algorithm for different supports",
)
ax.set_yscale("log")
plt.show()

### 5.4.  Interpret the results – state observations, limitations, advice regarding the use of the two algorithms!

In [ ]:
table_1.to_parquet(os.path.join(path_to_data, "table_1.parquet"), engine="pyarrow")

In [ ]:
table_2.to_parquet(os.path.join(path_to_data, "table_2.parquet"), engine="pyarrow")

---
<a id='ref'></a>

## References

<p> [1] http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions